# たし算
量子コンピュータには従来のコンピュータと同じ計算もできるという特徴があります。たし算の回路を確認します。

## 今回学ぶこと
1. 量子ゲートを使って二進数のたし算を実装
2. 量子の重ね合わせを使って1つの回路で複数のたし算

## Blueqatのインストール
pipからBlueqatをインストールします。

In [2]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## 二進数のたし算
たし算は桁上がりを実行するccxゲートと、位の足し合わせを実行するcxゲートを使います。今回はa+b=cdという二進数のたし算の量子回路を行います。今回はaとbの値によって4種類のたし算を実装します。それぞれのたし算は、

0+0 = 00 => 0000  
0+1 = 01 => 0101  
1+0 = 01 => 1001  
1+1 = 10 => 1110  

となります。4量子ビットのビット列で表現し、前半の2量子ビットが入力値aとbで、後半の2量子ビットが出力値cとdです。aとbを入力する回路と、実際にたし合わせを実現する回路を別に作り、何度か使い回します。1のようなデータの入力はXゲートを使って0を反転させて実装します。

たし算の回路部分はこのようになります。*はコントロールビットです。

```
a ---*---*------- a
b ---*---|---*--- b
0 ---X---|---|--- c
0 -------X---X--- d
```

aとbにXゲートを使ってデータを入れると、ccx回路で桁上がり、cx回路で位のたし合わせを行います。

In [6]:
#ツールの読み込み
from blueqat import Circuit

#たし算部分を実現します。
adder = Circuit().ccx[0,1,2].cx[0,3].cx[1,3]

In [13]:
#0+0
(Circuit() + adder).m[:].run(shots=100)

Counter({'0000': 100})

In [14]:
#0+1
(Circuit().x[1] + adder).m[:].run(shots=100)

Counter({'0101': 100})

In [15]:
#1+0
(Circuit().x[0] + adder).m[:].run(shots=100)

Counter({'1001': 100})

In [16]:
#1+1
(Circuit().x[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 100})

このように、たし算が実装できました。

## 重ね合わせを利用したたし算
ここで、Xゲートでデータを一つ一つ入れる代わりに、Hゲートを使ってたし算をしてみます。

In [17]:
#Xゲートの代わりにHゲートを入力に使う
(Circuit().h[0,1] + adder).m[:].run(shots=100)

Counter({'0101': 27, '1110': 30, '0000': 24, '1001': 19})

アダマールゲートを利用すると4つのたし算が大体1/4ずつ答えとして出てきました。このように汎用たし算回路を作ると、重ね合わせ状態を利用した計算を実行できます。

## もつれを利用したたし算
次にHゲートの代わりにa+b=1となるたし算を量子のもつれを使って行ってみます。

In [19]:
#01と10のもつれを作る
(Circuit().h[0].cx[0,1].x[0] + adder).m[:].run(shots=100)

Counter({'0101': 41, '1001': 59})

このように、入力値が01と10がもつれているので、この2つのたし算が約1/2ずつ出てきます。

--------

## 解説：回路の作成（１桁目）
まずは上の1桁目から作成します。
1桁目を見ると、0, 0, 0, 1 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの2桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|

これを見ると CCXゲートと同じことがわかります。
(CCXは入力の頭2つのビットが1ならば3番目のビットを反転させる)

このことから入力が 11 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_0.png" width="23%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（２桁目）
次にもう一つのくらいを作成します。
2桁目を見ると、0, 1, 1, 0 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの1桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

これを見ると CXゲートと同じことがわかります。
(左辺がCXの入力で右辺が出力の2番目のビット)

このことから入力が 10 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_1.png" width="27%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（全体）
最後に上の二つの回路をまとめます。

入力を 00 としてまとめた回路を以下に示します。

<img src="./img/008/008_02_2.png" width="30%">

上2ビットを入力、残りの2ビットが出力です。
始めの CCX は1桁目の部分で残りの 2つの CX は2桁目の部分です。

## 解説：重ね合わせ回路の実装
量子の重ね合わせを用いると4パターンを一気に操作することができます。

式としては以下の物を考えます。

<img src="./img/009/009_02_0.png" width="35%">

見てわかる通り右辺に入力の4パターンが出てきました。
この性質を用いて実装します。

回路は以下のようになります。

<img src="./img/009/009_02_1.png" width="35%">

今まで Xゲートを施していた部分を Hゲートにして 4パターンを作っています。

## (応用) 一般の足し算
一般の10進数同士で足し算を実装します。  
$a, b$ の和を考えることとします。
$a, b$ は $a = a_n ... a_0$, $b = b_n ... b_0$ と2進数で表示できます。  
(ここで n は数が大きい方の位を基準とします。)

回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

$c_i$ を carry ビットと言って繰り上がった数を表します。  
足し算回路は carry と呼ばれる繰り上がりの部分と sum の合計の部分にわかれています。

まずはこの2つの部分を考察します。

### 繰り上がり
回路は以下のようになります。

<img src="./img/100_0.png" width="30%">

回路を上から $c_i, a_i, b_i, c_{i+1}$ とすると $c_{i+1}$ に繰り上がりの部分が現れます。

### 合計
各位の和を考えます。回路は以下のようになります。

<img src="./img/100_1.png" width="30%">

回路を上から $c_i, a_i, b_i$ とすると $b_i$ に3つの数の和の繰り上がりを除いた部分が現れます。

### 実装
足し算回路を実装します。流れは以下のようになります。  

1. carry 回路で各位の繰り上がりを求める。  
2. CXゲートで最後のcarryの部分を元に戻す。  
3. sum 回路で $b_n$ に各位の和を出力する。  
4. carry の逆回路でその位の値を元に戻す。  
5. sum 回路で $b_i$ に各位の和を出力する。  
6. 4,5の繰り返し。

以上の流れから $a+b$ は $b_{n+1} ... b_0$ の部分に出力されることが分かります。

足し算回路を実装するために下準備をします。  
まずは carry とその逆回路, sum 回路を作ります。

In [2]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum(i):
    return Circuit().cx[i+1,i+2].cx[i,i+2]

10進数を2進数にする関数も作っておきます。

In [3]:
def tobinary(A):
    return bin(A)[2:]

tobinary(10)

'1010'

数を回路にマッピングする関数を作ります。

In [4]:
def digits(a,b): 
     # 2進数に変換
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # nを決めて大きい方にビット数を合わせる
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # マッピング
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

digits(2,2)

'0000110'

回路の初期状態は全て0なので、マッピングした値に合うようにXゲートを施す必要があります。  
この関数を作ります。

In [5]:
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

toX("101").m[:].run(shots=100)

Counter({'101': 100})

最後に出力の部分を考えます。  
まずは出力は2進数なので10進数に変換する関数を考えます。

In [6]:
def todecimal(A):
    return int(str(A),2) 

todecimal(1001)

9

回路では $a_i, b_i, c_i$ が混ざった値が出力されるのでその中で $b_i$ のみ取り出します。

In [7]:
def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

getb("0000110")

2

### 一般化回路
以上で全ての準備が整ったので足し算回路を作ります。

In [8]:
def plus(a,b): 
     # 2進数表示のマッピング
     qubits = len(digits(a,b)) 
     cir1 = toX(digits(a,b)) 
     digi = int((len(digits(a,b))-1)/3) 

     # 前半のcarry回路
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     # 最後の桁の処理
     cir3 = Circuit(qubits).cx[-3,-2] + sum((digi-1)*3) 

     # carryの逆回路とsum回路でbiに和を出力
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     result = (cir1 + cir2 + cir3 + cir4).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

実際に計算してみます。

In [19]:
plus(2,2)

4

In [20]:
plus(13,15)

28

In [21]:
plus(70,90)

160

最後の計算は時間かかりますが、一般の加算器を実装できました。

### 参考文献
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf

# ひき算
量子コンピュータには従来のコンピュータと同じ計算もできるという特徴があります。ひき算の回路を確認します。

## 今回学ぶこと
1. 量子ゲートを使って二進数のひき算を実装
2. 量子の重ね合わせを使って1つの回路で複数のひき算

## 二進数のひき算
たし算は符号を判定するccxゲートと、位の足し合わせを実行するcxゲートを使います。今回はa-b=cdという二進数のひき算の量子回路を行います。今回はaとbの値によって4種類のひき算を実装します。それぞれのひき算は、

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

となります。4量子ビットのビット列で表現し、前半の2量子ビットが入力値aとbで、後半の2量子ビットが出力値cとdです。aとbを入力する回路と、実際に引き算を実現する回路を別に作り、何度か使い回します。1のようなデータの入力はXゲートを使って0を反転させて実装します。

ひき算の回路部分はこのようになります。*はコントロールビットです。

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

aとbにXゲートを使ってデータを入れると、ccx回路で符号の判定、cx回路で位の引き算を行います。

In [1]:
#ツールの読み込み
from blueqat import Circuit

#ひき算部分を実現します。
substractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [2]:
#0-0
(Circuit() + substractor).m[:].run(shots=100)

Counter({'0000': 100})

In [3]:
#0-1
(Circuit().x[1] + substractor).m[:].run(shots=100)

Counter({'0111': 100})

In [4]:
#1-0
(Circuit().x[0] + substractor).m[:].run(shots=100)

Counter({'1001': 100})

In [5]:
#1-1
(Circuit().x[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 100})

このように、ひき算が実装できました。

## 重ね合わせを利用したひき算
ここで、Xゲートでデータを一つ一つ入れる代わりに、Hゲートを使ってひき算をしてみます。

In [6]:
#Xゲートの代わりにHゲートを入力に使う
(Circuit().h[0,1] + substractor).m[:].run(shots=100)

Counter({'0111': 32, '0000': 20, '1001': 25, '1100': 23})

アダマールゲートを利用すると4つのひき算が大体1/4ずつ答えとして出てきました。このように汎用ひき算回路を作ると、重ね合わせ状態を利用した計算を実行できます。

## もつれを利用したひき算
次にHゲートの代わりにa-b=0となるたし算を量子のもつれを使って行ってみます。

In [7]:
#00と11のもつれを作る
(Circuit().h[0].cx[0,1] + substractor).m[:].run(shots=100)

Counter({'1100': 48, '0000': 52})

このように、入力値が00と11がもつれているので、この2つのひき算が約1/2ずつ出てきます。

--------

## 解説：回路の作成（符号）
まずは上の符号から作成します。
符号は、0, 1, 0, 0 の順になっています。

表で表すと以下になります。

|X|Y|符号|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|0|
|1|1|0|

次にXの桁を反転させると

|X|Y|符号|
|:-:|:-:|:-:|
|1|0|0|
|1|1|1|
|0|0|0|
|0|1|0|

これを見ると CCXゲートと同じことがわかります。
(CCXは入力の頭2つのビットが1ならば3番目のビットを反転させる)

このことから入力が 11 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_0.png" width="23%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（２桁目）
次にもう一つのくらいを作成します。
2桁目を見ると、0, 1, 1, 0 の順になっています。

表で表すと以下になります。

|X|Y|X+Yの1桁目|
|:-:|:-:|:-:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

これを見ると CXゲートと同じことがわかります。
(左辺がCXの入力で右辺が出力の2番目のビット)

このことから入力が 10 の場合は以下の回路を考えることができます。

<img src="./img/008/008_02_1.png" width="27%">

上から2ビットを入力、残りの1ビットを出力だと考えると上の回路から 1 が出力されるのがわかります。

## 解説：回路の作成（全体）
最後に上の二つの回路をまとめます。

入力を 00 としてまとめた回路を以下に示します。

<img src="./img/008/008_02_3.png" width="40%">

上2ビットを入力、残りの2ビットが出力です。
始めの CCX は符号の部分で残りの 2つの CX は引き算の部分です。   
符号を検知するためにXゲートを施したのでCCXした後は再びXをして戻しています。

## 解説：重ね合わせ回路の実装
量子の重ね合わせを用いると4パターンを一気に操作することができます。

式としては以下の物を考えます。

<img src="./img/009/009_02_0.png" width="35%">

見てわかる通り右辺に入力の4パターンが出てきました。
この性質を用いて実装します。

回路は以下のようになります。

<img src="./img/009/009_02_2.png" width="40%">

今まで Xゲートを施していた部分を Hゲートにして 4パターンを作っています。

## (応用) 一般の引き算
一般の10進数同士で引き算を実装します。  
引き算回路は足し算回路を逆にすることで実装できます。

足し算回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

右から回路を考えます。
$a, a+b$ を入力して $b$ を出力します。 $a = a_n ... a_0$, $(a+b) = b_n ... b_0$ と2進数で表示できます。  
(ここで n は数が大きい方の位を基準とします。)

$c_i$ は繰り下がりを表します。  
この回路は carry と呼ばれる繰り上がりの部分と sum の合計の部分にわかれています。
まずはこの2つの部分を考察します。

### 繰り下がり
carry, sum は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F00361854-e782-0da0-3386-f59f668a1ada.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=58d1d41b0893b16bfb24bfd546328ff2" width="60%">

回路の始めの部分はこれらを組み合わせて繰り下がりを求めています。

### オバーフロービット
$a < a+b$ の場合でしか引き算はできません。   
判別を $b_{n+1}$ で行います。これをオーバーフロービットと言います。

$a < a+b$ のとき $b_{n+1}$ = 0  
$a > a+b$ のとき $b_{n+1}$ = 1 になります。

$a > a+b$ のときは $2^{n+1} - b$ が出力されます。

### 実装
引き算回路を実装します。

流れは以下のようになります。  
1. carry の逆回路と sum 回路で各位の繰り下がりを求める。  
2. オーバーフローを判別して $b_{n+1}$ に格納する。  
3. carry 回路で差を出力する。    

以上の流れから $b$ は $b_n ... b_0$ の部分に出力されることが分かります。
$b_{n+1}$ はオーバーフロービットです。

実装するために下準備をします。  
これは足し算回路と同じなので説明は省略します。

In [8]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

def tobinary(A):
    return bin(A)[2:]

def digits(a,b): 
     # 2進数に変換
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # nを決めて大きい方にビット数を合わせる
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # マッピング
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

def todecimal(A):
    return int(str(A),2) 

def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

In [9]:
def minus(a,ab): 
     # 入れ替え
     c = ab
     ab = a
     a = c

     # 2進数表示のマッピング
     qubits = len(digits(a,ab)) 
     cir1 = toX(digits(a,ab)) 
     digi = int((len(digits(a,ab))-1)/3) 

     # 前半のcarry回路とsum逆回路
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     # 最後の桁の処理
     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[-3,-2] 

     # carryの逆回路でbiに差を出力
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3)

     result = (cir1 + cir4 + cir3 + cir2).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

In [10]:
minus(8,2)

6

In [11]:
minus(4,2)

2

In [12]:
minus(50,24)

26

計算できました。ちなみに $a > a + b$ のときは

In [13]:
minus(2,4)

14

こちらもきちんと計算されています。

### 参考文献
https://arxiv.org/pdf/quant-ph/9511018.pdf

# かけ算
今回は二進数同士のかけ算を見てみましょう。

## 今回学ぶこと
1. 二進数のかけ算について
2. 回路を作成

## 二進数の掛け算について
２つの数をくらいごとにかけてずらして足し合わせます。その際に量子ビットを利用して桁上がりを考慮します。

0 * 0 = 0  
0 * 1 = 0  
1 * 0 = 0  
1 * 1 = 1  

これはccxゲートの挙動に対応しています。ccxゲートを利用してかけ算を行い、その後各位を足し合わせます。

## 例題
まずは例題を行います。01 * 10 = ?をときます。答えは、01 * 10 = 0010 となります。これを量子回路を利用して解きます。


``` 

   01   a
*  10   b
-------
   00   c
  01    c
-------
  0     z
 0      z
-------
 0010   x
```

では、早速実装へ。まずは2進数の数を２つ用意します。a * bを考えますが、aの0のくらいとaの2の位を用意して、それぞれa0とa2とします。bも同様です。

今回最終的に実現するのは|a,b,x> => |a, b, a * b >とします。求めたいのはx0,x2,x4,x8です。cは途中の計算用のビット。zは桁上がりビットを想定します。

<img src="./img/010_basic_multi01.png">


In [2]:
from blueqat import Circuit

C1 = Circuit().ccx[0,1,2].ccx[1,3,5].ccx[0,4,6].ccx[3,4,7].ccx[5,6,8].ccx[7,8,9].cx[2,10].cx[5,11].cx[6,11].cx[7,12].cx[8,12].cx[9,13] 

#00 * 00 = 0000
(Circuit() + C1).m[:].run(shots=100)

Counter({'00000000000000': 100})

In [3]:
#01 * 01 = 0001
(Circuit().x[0,1] + C1).m[:].run(shots=100)

Counter({'11100000001000': 100})

In [4]:
#10 * 01 = 0010
(Circuit().x[3,1] + C1).m[:].run(shots=100)

Counter({'01010100000100': 100})

In [5]:
#01 * 10 = 0010
(Circuit().x[0,4] + C1).m[:].run(shots=100)

Counter({'10001010000100': 100})

In [6]:
#10 * 10 = 0100
(Circuit().x[3,4] + C1).m[:].run(shots=100) 

Counter({'00011001000010': 100})

In [7]:
#11 * 10 = 0110
(Circuit().x[0,3,4] + C1).m[:].run(shots=100)

Counter({'10011011000110': 100})

In [8]:
#10 * 11 = 0110
(Circuit().x[1,3,4] + C1).m[:].run(shots=100)

Counter({'01011101000110': 100})

In [9]:
#11 * 11 = 1001
(Circuit().x[0,1,3,4] + C1).m[:].run(shots=100) 

Counter({'11111111111001': 100})

このように全ての掛け算のパターンが量子回路で実現できました。試しに入力を重ね合わせにしてみます。アダマールゲートをXゲートの代わりに入れると、

In [10]:
(Circuit().h[0,1,3,4] + C1).m[:].run(shots=100) 

Counter({'01010100000100': 7,
         '00001000000000': 5,
         '01000000000000': 6,
         '10010000000000': 6,
         '00011001000010': 12,
         '10011011000110': 9,
         '11100000001000': 3,
         '10000000000000': 3,
         '11101010001100': 9,
         '11110100001100': 5,
         '10001010000100': 9,
         '01001000000000': 8,
         '00010000000000': 3,
         '00000000000000': 6,
         '11111111111001': 4,
         '01011101000110': 5})

上手い感じに 00 * 00 から 11 * 11 までを計算することができました。

# 概要

## 乗算器
ビットは 0, 1 をとるので 二進数のかけ算を考えます。
組み合わせは

0×0=0, 0×1=0, 1×0=0, 1×1=1

となります。
これは加算器の1桁目と同じccxゲートで表せます。

## 乗算器 (2桁×2桁)
次は応用で2桁×2桁の乗算を考えてみます。

組み合わせは

01×00=0000, 01×01=0001, 01×10=0010, 01×11=0011   
11×00=0000, 11×01=0011, 11×10=0110, 11×11=1001   
01×00=0000, 01×01=0001, 01×10=0010, 01×11=0011   
11×00=0000, 11×01=0011, 11×10=0110, 11×11=1001

の16通りとなります。

### 考え方
まず試しに 11 × 10 を筆算で考えてみます。

<img src="./img/010_02/010_02_0.png" width="15%">

1 と 2 の行を合わせた4回の計算をそれぞれ考えます。   
1行目 は 11 × 0 , 2行目は 11 × 1 となります。   
各項4つの計算はそれぞれ 1桁の積でできるので全て CCXで実装できます。

行目1 と 2 の和に関しては繰り上がりを考えないといけないので加算器で実装します。

## 回路の作成（１、２行目）
今回は 1 行目だけ考えます。
11 × 0 を CCX ゲートを用いて以下の回路を考えます。

<img src="./img/010_02/010_02_1.png" width="28%">

上から 2 つのビットは 11 の部分で次の 1 ビットは 0 の部分となります。   
出力_1 ,出力_2 は 1 × 0 を表しています。

これで 1 行目が出力されました。2 行目も同様にしてできます。

## 回路の作成（全体）
11 × 10 を計算します。
上の2つの回路を組み合わせて以下の回路を作ります。

<img src="./img/010_02/010_02_3.png" width="65%">

左上の操作は 1, 2 行目を計算しています。
右下は 1, 2 行目の和を計算しています。

# 乗除演算
一般の足し算と引き算の仕組みを利用してあまりを求めようと思います。

回路は以下のようになります。

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Ff161e722-f727-b6fb-d02b-24e40b5b63b1.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=da298eace648814b5ccfe4affa9301ca">

ここで $0 < a,b < N$ とします。
最後のビットは overflow を確認するもので temporary bit と呼ばれます。

## 実装手順
$a+b \mod N$ を求めるためには $a+b$ と $N$ の大小を比較する必要があります。

$a+b>N$ のとき  
$0<a,b<N$ より $0<a+b<2N$ よって $0<a+b-N<N$ より $a+b-N = (a+b) \mod N$

$a+b<N$ のとき  
$a+b = (a+b) \mod N$

これを量子回路で行います。

$a+b>N$

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F5c97b6fd-425b-c42b-057a-68d9cb47e457.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=3711f239a543dc4aa175f4636adde888">

$a+b<N$

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2Fcb7bc1bf-e713-f7d4-9cdc-4bc47bb6eba8.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=425f2f86ef9d5b5c27b1c1f1ddfab936">

余剰の量子ビットと加算器の最上位の量子ビットの値を使ってうまく場合分けをしています。a+b<N の場合には余剰ビットを使わず余計な操作もありません。

## 例題
簡単な例題を見てみます。

$3 + 5 < 7$ のとき  
$(3 + 5) \mod 7 = 1$

$3 + 5 > 11$ のとき  
$(3 + 5) \mod 11 = 8$

これを量子回路を使って実現しようというのが今回の剰余演算です。

### 実装
実装するために下準備をします。

今回は上の回路に N をもう１つ加えて temporary bit と N を対応させます。  
初期状態を以下のようにします。

```
c0 --
a0 --
b0 --
c1 --
..
n0 --
n1 --
..
t  --
n0 --
n1 --
..
```

また各桁は $0<a,b<N$ より $N$ に統一します。 

In [11]:
from blueqat import Circuit

#ビットのキャリー回路
def carry(a):
    return Circuit().ccx[a+1,a+2,a+3].cx[a+1,a+2].ccx[a,a+2,a+3]

#ビットのキャリー回路の逆
def carry_reverse(a):
    return Circuit().ccx[a,a+2,a+3].cx[a+1,a+2].ccx[a+1,a+2,a+3]

#ビットの合計
def sum(a):
    return Circuit().cx[a+1,a+2].cx[a,a+2] 

#ビットの合計の逆
def sum_reverse(a):
    return Circuit().cx[a,a+2].cx[a+1,a+2]

#10進数を2進数に
def tobinary(A):
    return bin(A)[2:] 

#3つの10進数を2進数に直して、桁を揃えてモジュロ回路の順にビットを並べ替える。一番下に判定用のビットを１つ加える。
def digits2(a,b,n): 
     aa = tobinary(a)  
     bb = tobinary(b)  
     nn = tobinary(n)  

     nlen = len(nn)  
     aa = aa.zfill(nlen) 
     bb = bb.zfill(nlen) 

     str = '' 
     for i in range(nlen): 
         str += '0' + aa[nlen-i-1] + bb[nlen-i-1] 
     str += '0' 

     for i in range(nlen): 
         str += nn[nlen-i-1]  

     str += '0'

     for i in range(nlen): 
        str += nn[nlen-i-1]

     return str

#ビット文字列をXゲートを使ったデータ入力回路に変換
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

#足し算回路
def plus(a,b,n): 
     qubits = len(digits2(a,b,n))
     digi = len(tobinary(n))

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     cir3 = Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] + sum((digi-1)*3)

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     cir_plus = cir2 + cir3 + cir4
     return cir_plus

#引き算回路
def minus(a,ab,n): 
     qubits = len(digits2(a,ab,n))
     digi = len(tobinary(n))

     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += sum_reverse(i*3)
         cir4 += carry(i*3) 

     cir3 = sum_reverse((digi-1)*3) + Circuit(qubits).cx[(digi-1)*3+1,(digi-1)*3+2] 

     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry_reverse((digi-1-i)*3) 

     cir_minus = cir4 + cir3 + cir2
     return cir_minus

#aとNを交換
def swap(n):
    digi = len(tobinary(n))
    cir = Circuit(5*digi+2)
    for i in range(digi):
        cir += Circuit(5*digi+2).swap[3*i+1,3*digi+1+i]
    return cir

#2進数を10進数に
def todecimal(A):
    return int(str(A),2) 

#回路から解だけを抜き出す
def getb(result,n): 
     str = ""
     digi = len(tobinary(n)) 
     for i in range(digi): 
         str += result[3*(digi-i)-1]
     return todecimal(str)

### 一般化回路
以上で全ての準備が整ったので剰余演算回路を作ります。

In [12]:
def adder_mod(a,b,n):
    digi = len(tobinary(n))

    # 最初の部分
    part1 = toX(digits2(a,b,n)) + plus(a,b,n) + swap(n) + minus(a,b,n)

    # overflow を temporary bit に格納
    part2 = Circuit(5*digi+2).x[digi*3].cx[digi*3,digi*4+1].x[digi*3]

    # temporary bit で N を返す
    part3 = Circuit(5*digi+2)
    for i in range(digi):
        part3 += Circuit(5*digi+2).ccx[4*digi+2+i,4*digi+1,3*i+1]

    # 最後の部分
    part4 = minus(a,b,n)+Circuit(5*digi+2).cx[digi*3,digi*4+1]+plus(a,b,n)
    
    result = (part1+part2+part3+plus(a,b,n)+part3+swap(n)+part4).m[:].run(shots=1)
    return getb(result.most_common()[0][0],n)

実際に計算してみます。

In [13]:
adder_mod(4,3,5)

2

In [14]:
adder_mod(4,4,5)

3

In [15]:
adder_mod(1,5,6)

0

一般の剰余演算回路を実装できました。

### 参考文献
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf